In [2]:
# Dependencies
import googlemaps
import pandas as pd

In [ ]:
gmaps = googlemaps.Client(key="SECRET")

In [ ]:
bigfoot_df = pd.read_json('../data/filtered_years_clean.json')

In [ ]:
def geocode_with_fallback(row):
    try:
        # Try nearest town, state
        location = gmaps.geocode(f"{row['nearest town']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # Fallback to county, state
        location = gmaps.geocode(f"{row['county']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # If neither works, return None
        return None, None
    except Exception as e:
        print(f"Error geocoding {row['nearest town']}, {row['state']} or {row['county']}, {row['state']}: {e}")
        return None, None

In [ ]:
bigfoot_stub = bigfoot_df.iloc[:10]

In [ ]:
bigfoot_df[['Latitude', 'Longitude']] = bigfoot_df.apply(
    lambda row: pd.Series(geocode_with_fallback(row)), axis=1
)

In [ ]:
bigfoot_df = bigfoot_df.drop(columns='Unnamed: 0')
bigfoot_df.to_json('../data/bigfoot_coords_df.json')

In [9]:
bigfoot_coords = pd.read_json('../data/bigfoot_coords_df.json')

bigfoot_coords.columns

Index(['Report Number', 'Report Class', 'year', 'season', 'month', 'state',
       'county', 'location details', 'nearest town', 'nearest road',
       'observed', 'also noticed', 'other witnesses', 'other stories',
       'time and conditions', 'environment', 'date', 'a & g references',
       'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
final_columns = ['Report Number', 'Report Class', 'state', 'county', 'Latitude', 'Longitude', 'season', 'month', 'observed']

clean_coords_list = bigfoot_coords[final_columns]
clean_coords_list.to_json('../data/bigfoot_coordinates_clean_cols.json')